In [ ]:
# default_exp manage_data

# Manage Data And Model

In [ ]:
#export
from swfd.load_model import getAllHorizonPrediction
import datetime
import numpy as np
from swfd.resources import getInfo
import csv
import pandas as pd
import pickle
NORMALIZE=707.6
HORIZONS=[3,5,7,10,14,21,27]
COLUMNS=['Date','H3','H5','H7','H10','H14','H21','H27']

savePredict(): Guarda en un archivo todas las predicciones. En caso de existir remplaza el archivo.

In [ ]:
#export
def savePredict(datalist):
    path=getInfo("csvdirectory")+"predictionData.csv"
    try:
        outfile  = open(path, 'wb')
        pickle.dump(datalist, outfile )
    finally:
        outfile.close()


In [ ]:
#hide
testallpredict5=getAllHorizonPrediction(datetime.datetime(2021,5,20))
testallpredict6=getAllHorizonPrediction(datetime.datetime(2021,5,19))

df = pd.DataFrame(columns=COLUMNS)
df5 = pd.DataFrame (testallpredict5,columns=COLUMNS)
df6 = pd.DataFrame (testallpredict6,columns=COLUMNS)

df=df.append(df5)
df=df.append(df6)

df.reset_index(drop=True, inplace=True)
(df)
savePredict(df)

loadPredict(): Carga y devuelve todas las predicciones

In [ ]:
#export
def loadPredict():
    predict= pd.DataFrame(columns=COLUMNS)
    path=getInfo("csvdirectory")+"predictionData.csv"
    try:
        with open(path,'rb') as infile:
            predict = pickle.load(infile, encoding='bytes')
        infile.close() 
    except IOError:
        print("File not accessible")
        
    return predict

In [ ]:
#hide
loadPredict()

,Date,H3,H5,H7,H10,H14,H21,H27
0,2021-05-20,"[[81.21248168945313, 86.84930114746093, 90.730...","[[85.6896743774414, 91.3666488647461, 96.63803...","[[88.08998107910156, 91.18880462646484, 97.142...","[[86.47946166992188, 91.7086410522461, 99.8507...","[[98.36115264892578, 98.92632293701172, 103.43...","[[103.5103759765625, 98.6413803100586, 111.403...","[[100.23645782470703, 102.5992660522461, 106.1..."
1,2021-05-19,"[[79.1148452758789, 84.64676513671876, 89.8692...","[[84.84960632324218, 90.60640106201171, 97.151...","[[88.34898376464844, 91.91641998291016, 98.310...","[[85.222900390625, 91.44981384277344, 99.83406...","[[98.57737731933594, 100.28819274902344, 106.5...","[[102.22454833984375, 97.90068054199219, 110.4...","[[97.27412414550781, 100.40276336669922, 105.6..."


updatePredicts(): Hace todas las predicciones desde el primer día guardado del fichero de predicciones hasta hoy y las guarda en el fichero.

In [ ]:
#export
def updatePredicts():
    path=getInfo("csvdirectory")+"predictionData.csv"
    datapredict=loadPredict()
    if(not datapredict.empty): 
        yesterday = datetime.datetime.today().date()-datetime.timedelta(1)
        predictdate=pd.to_datetime(datapredict.loc[0,"Date"])+datetime.timedelta(1) #First date to predict
        
        if(yesterday >= predictdate):
        
            while(yesterday >= predictdate):# while yesterday is not in the past of data(predictdate)

                #print(predictdate,yesterday)
                print("Currently making ", predictdate.strftime("%Y-%m-%d") ," prediction")
                currentpredict=getAllHorizonPrediction(predictdate)
                datapredict=pd.concat([pd.DataFrame(currentpredict,columns=COLUMNS), datapredict], ignore_index=True)
                predictdate=predictdate+datetime.timedelta(1)
            
        
            print("Updated to ",yesterday)
            savePredict(datapredict)
        else:
            print("It was already updated")
    else:
        print("The file does not exist, you should make predictions to update")

In [ ]:
#hide
updatePredicts()

Currently making  2021-05-21 00:00:00  prediction


/home/miguel/anaconda3/lib/python3.8/site-packages/torch/nn/modules/container.py:435: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


Currently making  2021-05-22 00:00:00  prediction
Updated to  2021-05-22


updatePredicts(): Hace predicciones desde el último día guardado del fichero de predicciones 'days' veces y las guarda en el fichero.

In [ ]:
#export
def doNewPredicts(days):
    datapredict=loadPredict()
    
    
    if(not datapredict.empty):
        datestart=pd.to_datetime(datapredict.iloc[-1].Date)-datetime.timedelta(1)
    else:
        datestart=datetime.datetime.today().date()-datetime.timedelta(1)
    for numdays in range(days): 
        print((datestart-datetime.timedelta(numdays)).strftime("%Y-%m-%d"))
        currentpredict=getAllHorizonPrediction(datestart-datetime.timedelta(numdays))
        datapredict=pd.concat([datapredict,pd.DataFrame(currentpredict,columns=COLUMNS)], ignore_index=True)
        print("updated")
        savePredict(datapredict)

In [ ]:
#hide
doNewPredicts(2)
loadPredict()

2021-05-18 00:00:00
updated
2021-05-17 00:00:00
updated


,Date,H3,H5,H7,H10,H14,H21,H27
0,2021-05-22,"[[84.03729095458985, 88.03429107666015, 89.865...","[[84.67812957763672, 88.30326843261719, 92.606...","[[84.67823791503906, 88.69149780273438, 92.523...","[[80.38555908203125, 84.468994140625, 92.61737...","[[101.35305786132812, 99.46685028076172, 103.8...","[[103.991455078125, 99.47657775878906, 109.813...","[[102.07795715332031, 103.3031005859375, 103.4..."
1,2021-05-21,"[[83.69420166015625, 88.32374572753906, 91.195...","[[85.68935699462891, 90.75347900390625, 94.940...","[[88.5923843383789, 93.01776885986328, 96.3802...","[[85.0068130493164, 88.88204956054688, 96.9358...","[[99.32980346679688, 99.18195343017578, 103.22...","[[103.77058410644531, 97.80664825439453, 110.7...","[[101.01580047607422, 102.25955200195312, 104...."
2,2021-05-20,"[[81.21248168945313, 86.84930114746093, 90.730...","[[85.6896743774414, 91.3666488647461, 96.63803...","[[88.08998107910156, 91.18880462646484, 97.142...","[[86.47946166992188, 91.7086410522461, 99.8507...","[[98.36115264892578, 98.92632293701172, 103.43...","[[103.5103759765625, 98.6413803100586, 111.403...","[[100.23645782470703, 102.5992660522461, 106.1..."
3,2021-05-19,"[[79.1148452758789, 84.64676513671876, 89.8692...","[[84.84960632324218, 90.60640106201171, 97.151...","[[88.34898376464844, 91.91641998291016, 98.310...","[[85.222900390625, 91.44981384277344, 99.83406...","[[98.57737731933594, 100.28819274902344, 106.5...","[[102.22454833984375, 97.90068054199219, 110.4...","[[97.27412414550781, 100.40276336669922, 105.6..."
4,2021-05-18,"[[77.24601287841797, 82.04307403564454, 87.805...","[[82.9322738647461, 89.56397552490235, 95.4848...","[[89.31147003173828, 91.42094421386719, 97.182...","[[84.14317321777344, 91.30431365966797, 99.914...","[[96.60798645019531, 98.28929138183594, 107.74...","[[97.37294006347656, 94.68815612792969, 106.97...","[[93.70542907714844, 95.90367889404297, 101.94..."
5,2021-05-17,"[[69.76936645507813, 74.02055511474609, 79.464...","[[75.6918441772461, 82.5073471069336, 88.62305...","[[81.01795196533203, 80.90645599365234, 90.064...","[[81.79053497314453, 88.2459487915039, 97.7942...","[[91.79501342773438, 90.76585388183594, 102.00...","[[89.51701354980469, 88.32717895507812, 99.790...","[[87.9896469116211, 89.56783294677734, 97.1685..."


infoDatesPredicts(): Devuelve los días entre los que comprende el fichero de datos.

In [ ]:
#export
def infoDatesPredicts():
    startdate=None
    enddate=None
    datapredict=loadPredict()
    if(not datapredict.empty): 
        startdate=pd.to_datetime(datapredict.iloc[0].Date)
        enddate=pd.to_datetime(datapredict.iloc[-1].Date)
    return startdate,enddate

In [ ]:
#hide
infoDatesPredicts()

(Timestamp('2021-05-22 00:00:00'), Timestamp('2021-05-17 00:00:00'))